## New .NET Interactive Features

In [1]:
var hello = "Hello";

In [1]:
#!fsharp
#!share --from csharp hello

let world = "World"

open System

Console.Write($"{hello}, {world}");

Hello, World

In [1]:
#!value --name users
{
    "id": "1",
    "name": "Jon"
}

In [1]:
class Users
{
    public string Id { get; set; }
    public string Name { get; set; }
}

In [1]:
using Newtonsoft.Json;

#!share users --from value

var obj = JsonConvert.DeserializeObject<Users>(users);

Console.Write($"{obj.Id}, {obj.Name}");

1, Jon

In [1]:
#!value --from-url https://github.com --name site

In [1]:
#!share --from value site

display(site, "text/html")

## ML.NET Sample

In [1]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.Data.Analysis"

Installed package Microsoft.Data.Analysis version 0.4.0

Installed package Microsoft.ML version 1.5.2

In [1]:
using Microsoft.ML;
using Microsoft.Data.Analysis;

var context = new MLContext();

In [1]:
var data = DataFrame.LoadCsv("../Data/bank.csv", header: true, separator: ';');

In [1]:
data.Preview().ColumnView

index,Column,Values
0,age: Single,"[ 30, 33, 35, 30, 59, 35, 36, 39, 41, 43, 39, 43, 36, 20, 31, 40, 56, 37, 25, 31 ... (80 more) ]"
1,job: String,"[ unemployed, services, management, management, blue-collar, management, self-employed, technician, entrepreneur, services, services, admin., technician, student, blue-collar, management, technician, admin., blue-collar, services ... (80 more) ]"
2,marital: String,"[ married, married, single, married, married, single, married, married, married, married, married, married, married, single, married, married, married, single, single, married ... (80 more) ]"
3,education: String,"[ primary, secondary, tertiary, tertiary, secondary, tertiary, tertiary, secondary, tertiary, primary, secondary, secondary, tertiary, secondary, secondary, tertiary, secondary, tertiary, primary, secondary ... (80 more) ]"
4,default: String,"[ no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no ... (80 more) ]"
5,balance: Single,"[ 1787, 4789, 1350, 1476, 0, 747, 307, 147, 221, -88, 9374, 264, 1109, 502, 360, 194, 4073, 2317, -221, 132 ... (80 more) ]"
6,housing: String,"[ no, yes, yes, yes, yes, no, yes, yes, yes, yes, yes, yes, no, no, yes, no, no, yes, yes, no ... (80 more) ]"
7,loan: String,"[ no, yes, no, yes, no, no, no, no, no, yes, no, no, no, no, yes, yes, no, no, no, no ... (80 more) ]"
8,contact: String,"[ cellular, cellular, cellular, unknown, unknown, cellular, cellular, cellular, unknown, cellular, unknown, cellular, cellular, cellular, cellular, cellular, cellular, cellular, unknown, cellular ... (80 more) ]"
9,day: Single,"[ 19, 11, 16, 3, 5, 23, 14, 6, 14, 17, 20, 17, 13, 30, 29, 29, 27, 20, 23, 7 ... (80 more) ]"


In [1]:
var stringColumns = data.Columns
    .Select(col => col.Name)
    .Where(c => !new[] { "age", "balance", "day", "duration", "campaign", "pdays", "previous" }.Contains(c))
    .ToArray();

In [1]:
var trainTestSplit = context.Data.TrainTestSplit(data, testFraction: 0.2);

In [1]:
var textOptions = new Microsoft.ML.Transforms.Text.TextFeaturizingEstimator.Options();

In [1]:
var pipeline = context.Transforms.Conversion.ConvertType("Label", "default", Microsoft.ML.Data.DataKind.Boolean)
    .Append(context.Transforms.Text.FeaturizeText("Text", textOptions, stringColumns))
    .Append(context.Transforms.Concatenate("Features", "age", "balance"))
    .Append(context.Transforms.Concatenate("Features", "Features", "Text"))
    .Append(context.BinaryClassification.Trainers.LbfgsLogisticRegression());

var model = pipeline.Fit(trainTestSplit.TrainSet);

In [1]:
var crossValidation = context.BinaryClassification.CrossValidate(trainTestSplit.TestSet, pipeline);

In [1]:
crossValidation.Average(cv => cv.Metrics.AreaUnderRocCurve)

0.902135441207083

In [1]:
class BankData
{
    public float age { get; set; }
    public string job { get; set; }
    public string marital { get; set; }
    public string education { get; set; }
    public string @default { get; set; }
    public float balance { get; set; }
    public string housing  { get; set; }
    public string loan { get; set; }
    public string contact { get; set; }
    public float day { get; set; }
    public string month { get; set; }
    public float duration { get; set; }
    public float campaign { get; set; }
    public float pdays { get; set; }
    public float previous { get; set; }
    public string poutcome { get; set; }
    public string y { get; set; }
}

class BankPrediction
{
    public bool PredictedLabel { get; set; }
    public float Score { get; set; }
}

In [1]:
var predictionEngine = context.Model.CreatePredictionEngine<BankData, BankPrediction>(model);

In [1]:
var newItem = new BankData
{
    age = 22,
    job = "entrepreneur",
    marital = "single",
    education = "primary",
    balance = -900,
    housing = "yes",
    loan = "yes",
    contact = "cellular"
};

var anotherItem = new BankData
{
    age = 44,
    job = "management",
    marital = "married",
    education = "secondary",
    balance = 4200,
    housing = "yes",
    loan = "no",
    contact = "cellular"
};

In [1]:
var prediction = predictionEngine.Predict(newItem);

prediction.PredictedLabel

True

In [1]:
var prediction1 = predictionEngine.Predict(anotherItem);

prediction1.PredictedLabel

False